In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

In [3]:
from helper.object_detection_helper import *
from loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from models.RetinaNet import RetinaNet
from tqdm import tqdm

from helper.nms_center_distance import non_max_suppression_by_distance

In [4]:
def get_y_func(x):
    return [[[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]], [1,2,3,4,5]]

In [5]:
path = Path('/data/Datasets/EIPH_WSI/RCNN-Patches/Inference/11_EIPH_575697 Berliner Blau')
bs = 4
data = (ObjectItemList.from_folder(path)
        #Where are the images? -> in coco
        .split_none()
        #How to split in train/valid? -> randomly with the default 20% in valid
        .label_from_func(get_y_func)
        #How to find the labels? -> use get_y_func
        .transform(get_transforms(), tfm_y=True, size=1024)
        #Data augmentation? -> Standard transforms with tfm_y=True
        .databunch(bs=bs, collate_fn=bb_pad_collate))\
    .normalize([[0.9296, 0.9177, 0.9290], [0.0986, 0.0931, 0.0679]])

In [6]:
anchors = create_anchors(sizes=[(32,32), (16,16), (8,8)], ratios=[1], scales=[0.6, 0.7, 0.9, 1.25, 1.5])
data.train_ds.classes

['background', 1, 2, 3, 4, 5]

In [7]:
crit = RetinaNetFocalLoss(anchors)
encoder = create_body(models.resnet50, True, -2)
model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=5, sizes=[8,16,32], chs=256, final_bias=-4., n_conv=4)
learn = Learner(data, model, loss_func=crit)#.load("/data/Datasets/EIPH_WSI/models/baseline-level0-rn18")

In [8]:
processed_images = 0
detect_thresh = 0.3
nms_thresh = 0.3
size = 1024

In [9]:
%%time
total_bbox_pred = []
total_class_pred = []
total_score_pred = []
for xb, yb in tqdm(learn.data.train_dl):
    file_names_batch = learn.data.train_ds.x.items[processed_images: processed_images + bs]
    x_offset_batch, y_offset_batch = list(zip(*[fn.stem.split('_')[:2] for fn in file_names_batch]))

    class_pred_batch, bbox_pred_batch = learn.model(xb)[:2] # [:, [2, 1, 0], :, :]

    for clas_pred, bbox_pred, x_offset, y_offset in zip(class_pred_batch, bbox_pred_batch, x_offset_batch, y_offset_batch):
        bbox_pred, scores, preds = process_output(clas_pred, bbox_pred, anchors, detect_thresh)
        if bbox_pred is not None:
            #to_keep = nms(bbox_pred, scores, nms_thresh)
            t_sz = torch.Tensor([size, size])[None].float()
            bbox_pred = to_np(rescale_boxes(bbox_pred.cpu(), t_sz))

            bbox_pred[:, 1] = bbox_pred[:, 1] + int(x_offset)
            bbox_pred[:, 0] = bbox_pred[:, 0] + int(y_offset)

            total_bbox_pred.extend(bbox_pred)
            total_class_pred.extend(to_np(preds))
            total_score_pred.extend(to_np(scores))


    processed_images += bs
    #print(processed_images)

100%|██████████| 1055/1055 [03:41<00:00,  4.94it/s]

CPU times: user 3min 9s, sys: 1min 9s, total: 4min 18s
Wall time: 3min 41s
